In [10]:
import nest_asyncio
nest_asyncio.apply()

In [12]:

from dataclasses import dataclass
import asyncio
from agents.extensions.visualization import draw_graph
import os 
from openai import AsyncOpenAI 
from dotenv import load_dotenv  
load_dotenv() 
from agents import (
    Agent,
    Runner, 
    set_tracing_disabled,OpenAIChatCompletionsModel,enable_verbose_stdout_logging
)   
 

# enable_verbose_stdout_logging()
set_tracing_disabled(disabled=True)
API_KEY=os.getenv("GEMINI_API_KEY")

if  not API_KEY  :
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

 
client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",api_key=API_KEY,)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash",openai_client=client)




@dataclass
class UserContext:
    text: str
    translated_text: str = ""    
    
    
def translate_instructions(wrappers,agent):
    return (
        "You are a helpful assistant. Translate this text into Urdu:\n\n"
        f"\"{wrappers.context.text}\""
    )
 
 
def story_instructions(wrapper, agent):
    return (
        "You are a creative story writer. Write a short story based on the Urdu text:\n\n"
        f"\"{wrapper.context.translated_text}\""
    )

async def main(user_input: str): 
    ctx = UserContext(text=user_input) 
    
    translator = Agent[UserContext](
        name="Urdu Translator",
        instructions=translate_instructions,
        model=model
    ) 
    
    trans_result = await Runner.run(
        starting_agent=translator,
        input=user_input,
        context=ctx
    ) 
    ctx.translated_text = trans_result.final_output.strip() 
    story_agent = Agent[UserContext](
        name="Story Generator",
        instructions=story_instructions,
        model=model
    ) 
 
    story_result = await Runner.run(
        starting_agent=story_agent,
        input=ctx.translated_text,
        context=ctx
    )
 
    print(f"original text : {ctx.text}")
    print("→ Urdu Translation:\n", ctx.translated_text)
    print("\n→ Generated Story:\n", story_result.final_output)
if __name__ == "__main__":
    
    text  = "allama iqbal is our national poet"

    asyncio.run(main(text ))

original text : allama iqbal is our national poet
→ Urdu Translation:
 Here's the translation of "Allama Iqbal is our national poet" into Urdu:

**علامہ اقبال ہمارے قومی شاعر ہیں۔**

(Allama Iqbal hamare qaumi shayar hain.)

→ Generated Story:
 The cool morning air, thick with the scent of jasmine from the nearby garden, ruffled the pages of old Fatima's Quran. She sat on the worn charpai in her courtyard, the sunlight painting dappled patterns on the mud walls. Inside, her grandson, young Imran, wrestled with his Urdu homework.

"Nani Amma," Imran groaned, his voice muffled by the textbook, "I just don't understand why we have to learn this *again*."

Fatima, her eyes twinkling with a life lived fully and lovingly, lowered her Quran slightly. "What troubles you, beta?"

"This sentence, Nani Amma! **علامہ اقبال ہمارے قومی شاعر ہیں۔** (Allama Iqbal hamare qaumi shayar hain.) Allama Iqbal is our national poet. I know, I know! But it feels...distant. Like something just in a textbook."

F